TITLE

Dataset from: https://archive.ics.uci.edu/ml/datasets/Wine

INTRO

We will be using the dataset "wine", from the url: https://archive.ics.uci.edu/ml/datasets/wine/wine.data

The "wine" dataset contains the data of 13 attributes of wines, created from the same region of Italy, created from 3 different cultivars (cultivar 1, 2, and 3). A cultivar is a plant or fruit variety that has been cultivated for specific characteristics through selectively breeding. There is no information on what type of plant/fruit these cultivars are, therefore there is not enough information about the dataset to determine whether these 3 cultivars are 3 distinct plants/fruits, or 3 varieties of the same fruit/plant. After a chemical analysis has been performed on the wine, 13 chemical attributes, such as alcohol content, color intensity, hue, and amount of magnesium content, have been determined. With each of these 13 attributes, we will be able to create a classification model which predicts which of the 3 cultivars a wine originates from based on it’s chemical breakdown, answering the question we have divised for our project, “What cultivar does this wine belong to, based on it’s chemical breakdown?”

PRELIMINARY EXPLORATORY DATA ANALYSIS

Demonstrate that the dataset can be read from the web into R 
Clean and wrangle your data into a tidy format
Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

METHODS

Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
Describe at least one way that you will visualize the results

EXPECTED OUTCOME AND SIGNIFICANCE

What do you expect to find?
What impact could such findings have?
What future questions could this lead to?